In [180]:
import pyspark

In [181]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("assignment") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [182]:
df1 = spark.read.parquet("C:/Users/Tejashree/Downloads/Redhat Assignment/consumerInternet.parquet")

In [183]:
df2 = spark.read.load("C:/Users/Tejashree/Downloads/Redhat Assignment/startup.csv", format="csv", sep=",", inferSchema="true", header="true")

In [184]:
df = df1.union(df2)

In [185]:
df.columns

['Sr_No',
 'Date',
 'Startup_Name',
 'Industry_Vertical',
 'SubVertical',
 'City',
 'Investors_Name',
 'InvestmentnType',
 'Amount_in_USD',
 'Remarks']

In [186]:
df.createOrReplaceTempView("startup")

## How many startups are there in Pune City?

In [187]:
spark.sql("SELECT count(distinct Startup_name) as total_startups FROM startup").show()

+--------------+
|total_startups|
+--------------+
|          2459|
+--------------+



## How many startups in Pune got their Seed/ Angel Funding?

In [188]:
spark.sql("SELECT count(distinct Startup_name) as total_startups FROM startup where lower(InvestmentnType) like '%seed%' or lower(InvestmentnType) like '%angel%' ").show()

+--------------+
|total_startups|
+--------------+
|          1435|
+--------------+



## What is the total amount raised by startups in Pune City? Hint - use regex_replace to get rid of null

In [189]:
spark.sql("select sum(regexp_replace(amount, '[^0-9]', 0)) as total_amount from (select CAST(regexp_replace(Amount_in_USD, ',', '') AS DOUBLE) as amount from startup where lower(city) = 'pune') where amount is not null").show()

+-------------+
| total_amount|
+-------------+
|9.788800999E9|
+-------------+



## What are the top 5 Industry_Vertical which has the highest number of startups in India?

In [191]:
spark.sql("select industry_vertical, count(distinct startup_name) as startups from startup where industry_vertical != 'nan' group by industry_vertical order by startups desc limit 5").show()

+-----------------+--------+
|industry_vertical|startups|
+-----------------+--------+
|Consumer Internet|     835|
|       Technology|     453|
|        eCommerce|     162|
|       Healthcare|      70|
|        ECommerce|      59|
+-----------------+--------+



## Find the top Investor(by amount) of each year.

In [192]:
spark.sql("""select investors_name, year, total_amount from (select investors_name, year, total_amount, rank() over (partition by year order by total_amount desc) as rank from 
            (select investors_name, year, sum(regexp_replace(amount, '[^0-9]', 0)) as total_amount from
                (select Investors_Name, year(TO_DATE(CAST(UNIX_TIMESTAMP(date, 'dd/MM/yyyy') AS TIMESTAMP))) as year, 
                        CAST(regexp_replace(Amount_in_USD, ',', '') AS DOUBLE) as amount 
                        from startup where date != '01/07/015' and investors_name != 'N/A') 
                where year is not null group by investors_name, year)) where rank = 1 order by year asc""").show()

+--------------------+----+---------------+
|      investors_name|year|   total_amount|
+--------------------+----+---------------+
|       SAIF Partners|2015|          1.4E9|
|     Kalaari Capital|2016|         1.24E9|
|Bennett, Coleman ...|2017|         1.55E9|
|Avendus Finance P...|2018|          9.7E8|
|             FinTech|2019|1.0741126507E10|
|Sequoia Capital I...|2020|  1.083588607E9|
+--------------------+----+---------------+



In [ ]:
## Find the top startup(by amount raised) from each city?

In [193]:
spark.sql("""select startup_name, city, amount_raised from (select startup_name, city, amount_raised, rank() over (partition by city order by amount_raised desc) as rank from 
            (select startup_name, city, sum(regexp_replace(amount, '[^0-9]', 0)) as amount_raised from
                (select startup_name, city, CAST(regexp_replace(Amount_in_USD, ',', '') AS DOUBLE) as amount 
                        from startup) group by  startup_name, city) where  amount_raised is not null) where rank <= 1""").show()

+--------------------+-----------------+-------------+
|        startup_name|             city|amount_raised|
+--------------------+-----------------+-------------+
|                Zefo|        Bangalore|       1.52E9|
|            Uniphore|         Taramani|    3080807.0|
|    Karma Healthcare|          Udaipur|        5.0E7|
|         Convegenius|Noida / Singapore|        3.0E7|
|        Lenskart.com|        Faridabad|     203108.0|
|Vogo Automotive P...|        Karnataka|      3.584E8|
|               GOQii|       Menlo Park|      40508.0|
|            Uniphore|        Palo Alto|      50107.0|
|         VST Travels|           Kerala|    2700000.0|
|               CBREX|      Mumbai / NY|        2.0E7|
|           Active.ai|        Singapore|        3.5E8|
|               Qriyo|          Jodhpur|        1.6E7|
|           Elucidata|Delhi & Cambridge|        1.7E8|
|           Pastiwala|         Vadodara|        4.0E8|
|             Mazkara|     Pune / Dubai|        1.0E8|
|        C